# <span style="font-size: 1.5em;">Análisis y modelado de datos</span>


En este notebook, abordaremos el proceso completo de análisis y modelado de datos relacionados con las resoluciones e informes de la AGN (Auditoría General de la Nación). El objetivo principal es automatizar y estandarizar el proceso de adquisición, limpieza, transformación y modelado de estos datos para obtener insights valiosos y posiblemente desarrollar un modelo predictivo o clasificatorio.

#### <span style="font-size: 1.5em;">Librerías</span>

In [1]:
import sqlite3  # Codigo 1 y 2
# Librería para trabajar con bases de datos SQLite.

from IPython.display import display, HTML  # Codigo 1
# Herramientas para mostrar y formatear la salida en entornos Jupyter.

import pandas as pd # Codigo 1 y 2
# Librería para manipulación y análisis de datos, especialmente para estructuras de datos tabulares.


#### <span style="font-size: 1.5em;">Funciones</span>

In [2]:
# Codigo 1

# Función para estilizar y mostrar DataFrames
def mostrar_dataframe_estilizado(df, num_head=None, num_tail=None):
    
    # Si num_head y num_tail son None, mostrar todo el DataFrame
    if num_head is None and num_tail is None:
        df_resumen = df
    else:
        # Si no, concatenar las filas del principio y del final según los valores de num_head y num_tail
        df_resumen = pd.concat([df.head(num_head), df.tail(num_tail)])
    
    styled_table = df_resumen.style.set_table_styles([
        {'selector': 'table',
         'props': [('border', '2px solid black'), ('border-collapse', 'collapse')]},
        {'selector': 'th',
         'props': [('background-color', 'lightgray'), ('border', '1px solid black'), ('padding', '5px')]},
        {'selector': 'td',
         'props': [('border', '1px solid black'), ('padding', '5px')]}
    ])
    
    # Mostrar el DataFrame estilizado
    display(styled_table)


#### <span style="font-size: 1.5em;">Descripción del Dataset</span>

El dataset generado proviene principalmente de la tabla "detalle_Informes" de la base de datos "informes_agn.db". La información del dataset se complementó con datos de otras tablas de la base de datos.

### Base de Datos

- **Nombre**: "informes_agn.db"

### Tablas Utilizadas

- **detalle_Informes**: Tabla principal de donde se extrajo la mayoría de la información.
- **codigo_archivos**: Tabla que contiene información sobre los archivos relacionados.
- **codigo_tid**: Tabla que contiene códigos y descripciones que se usaron para reemplazar y enriquecer la información.
- **codigo_infografias**: Tabla con información sobre las infografías relacionadas.
- **resolucion_informe_texto**: Tabla que contiene el texto de las resoluciones y de los informes.

Con la combinación de estas tablas, se logró obtener un dataset más completo y adecuado para análisis detallados.

## Dataset

### Columnas Principales

- **codigo_nid**: Identificador único del informe.
- **titulo**: Título del informe.
- **estado**: Categoriza los informes en "Informe reservado", "Difusión restringida" o "Disponible".
- **gerencia**: Gerencia responsable del informe.
- **tipo_de_auditoria**: Tipo de auditoría realizada.
- **organismo_auditado**: Organismo que fue auditado.
- **actuación**: Representa la actuación del informe, y se descompone en:
  - **cod_actuacion**: Código de actuación.
  - **ano_actuacion**: Año de actuación.
- **ano**: Año de aprobación del informe.
- **ventana_de_trabajo**: Tiempo de realización del informe.
- **periodo_start**: Comienzo del periodo auditado.
- **periodo_end**: Finalización del periodo auditado.
- **numero_resolucion**: Número de resolución de aprobación.
- **auditoria_coordinada**: Indica si la auditoría fue coordinada o no.

### Palabras Clave

Para las columnas derivadas de "cod_tid_contenido" a "codigo_tid", se sustituyeron los códigos originales por sus descripciones. Estas columnas representan palabras clave del informe:

- **ods**: Objetivos de Desarrollo Sostenible relacionados.
- **sector_publico_nacional**: Sector público nacional involucrado.
- **objeto_de_auditoria**: Objeto específico de la auditoría.
- **jurisdiccion**: Jurisdicción relacionada.
- **tags**: Palabras clave utilizadas en el filtro de la web.

### Disponibilidad de Documentos

Las siguientes columnas indican la disponibilidad de documentos o formatos vinculados al informe:

- **resolucion**
- **informe**
- **ficha**
- **anexo**
- **imagen**
- **informe_en_video**
- **informe_multimedia**
- **infografia**

In [3]:
# Codigo 2

# Conectar a la base de datos SQLite
conn = sqlite3.connect('informes_agn.db')

# Cargar las tablas en DataFrames
df_detalle_Informes = pd.read_sql_query("SELECT * FROM detalle_Informes", conn)
df_codigo_tid = pd.read_sql_query("SELECT * FROM codigo_tid", conn)
df_resolucion_informe_texto = pd.read_sql_query("SELECT * FROM resolucion_informe_texto", conn)

# Función para detectar la situación en el texto
def detect_situation(text):
    if text is None:
        return 'Disponible'
    
    situations_restringida = ['restringida', 'restringido', 'restricción']
    situations_reservado = ['reservado', 'reservada', 'reserva']
    
    for situation in situations_restringida:
        if situation in text:
            return "Difusión restringida"
    for situation in situations_reservado:
        if situation in text:
            return "Informe reservado"
    return 'Disponible'

# Crear la columna 'estado'
df_detalle_Informes = df_detalle_Informes.merge(df_resolucion_informe_texto[['codigo_nid', 'texto_resolucion']], on='codigo_nid', how='left')
df_detalle_Informes['estado'] = df_detalle_Informes.apply(lambda row: detect_situation(row['descripcion']) if detect_situation(row['descripcion']) != "Disponible" else detect_situation(row['texto_resolucion']), axis=1)

# Dividir la columna 'actuacion' en 'cod_actuacion' y 'ano_actuacion'
df_detalle_Informes['cod_actuacion'], df_detalle_Informes['ano_actuacion'] = zip(*df_detalle_Informes['actuacion'].apply(lambda x: x.split('/') if x and '/' in x else (None, None)))

# Convertir la columna 'ano_actuacion' a flotante, manejando valores de cadena vacía y None
df_detalle_Informes['ano_actuacion'] = df_detalle_Informes['ano_actuacion'].apply(lambda x: float(x) if x and x.strip() != '' else None)

# Calcular 'ventana_de_trabajo'
df_detalle_Informes['ventana_de_trabajo'] = df_detalle_Informes['ano'] - df_detalle_Informes['ano_actuacion']

# Función para encontrar una coincidencia aproximada en el texto
def find_approximate_match(text, gerencias):
    for gerencia in gerencias:
        keywords = gerencia.split()
        if sum([1 for keyword in keywords if keyword in text]) >= len(keywords) * 0.7:  # Si al menos el 70% de las palabras clave están presentes
            return gerencia
    return None

# Función para extraer gerencia de las columnas 'texto_resolucion' y 'texto_informe'
def extract_gerencia(row, gerencias):
    gerencia = find_approximate_match(row['texto_resolucion'], gerencias)
    if not gerencia:
        gerencia = find_approximate_match(row['texto_informe'], gerencias)
    return gerencia

# Extraer gerencia
gerencias = [
        "Gerencia de Control de Gestión del Sector No Financiero",
        "Gerencia de Control Financiero del Sector No Financiero",
        "Gerencia de Control del Sector Financiero y Recursos",
        "Gerencia de Control de Entes Reguladores y Empresas Prestadoras de Servicios Públicos",
        "Gerencia de Control de la Deuda Pública",
        "Gerencia de Control de Transferencias de Fondos Nacionales",
        "Gerencia de Control de la Cuenta de Inversión",
        "Gerencia de Planificación y Proyectos Especiales",
        "Gerencia de Administración y Finanzas",
        "Auditoria Interna",
        "Secretaria Legal e Institucional",
        "Prosecretaria Legal",
        "Departamento Legal",
        "Prosecretario Institucional",
        "Departamento de Relaciones Institucionales y Convenios",
        "Departamento de Prensa y Comunicación",
        "Gerencia de Coordinación",
        "Secretaría Ejecutiva de Presidencia",
        "Oficina de Actuaciones Colegiadas"
        ]
df_resolucion_informe_texto['gerencia'] = df_resolucion_informe_texto.apply(extract_gerencia, axis=1, args=(gerencias,))

# Merge para agregar la columna 'gerencia' al DataFrame principal
df_detalle_Informes = df_detalle_Informes.merge(df_resolucion_informe_texto[['codigo_nid', 'gerencia']], on='codigo_nid', how='left')

# Función para reemplazar códigos con su contenido correspondiente
def replace_with_tid_content(cod, tid_df):
    if pd.isnull(cod):
        return "no identificado"
    codes = str(cod).split(',')
    contents = []
    tid_set = set(tid_df['cod_tid'].values)
    for code in codes:
        try:
            int_code = int(code)
            if int_code in tid_set:
                contents.append(tid_df.set_index('cod_tid').loc[int_code]['cod_tid_contenido'])
            else:
                contents.append("no identificado")
        except ValueError:
            contents.append("no identificado")
    return ', '.join(contents)

# Procesar las columnas de códigos tid
tid_columns = ['cod_tid_tipo_de_auditoria', 'cod_tid_organismo_auditado', 'cod_tid_ods', 'cod_tid_sector_publico_nacional', 
               'cod_tid_objeto_de_auditoria', 'cod_tid_jurisdiccion', 'cod_tid_tags']
new_columns = ['tipo_de_auditoria', 'organismo_auditado', 'ods', 'sector_publico_nacional', 
               'objeto_de_auditoria', 'jurisdiccion', 'tags']

for tid_col, new_col in zip(tid_columns, new_columns):
    df_detalle_Informes[new_col] = df_detalle_Informes[tid_col].apply(replace_with_tid_content, args=(df_codigo_tid,))

# Procesar las columnas de códigos fid
fid_columns = ['cod_fid_resolucion', 'cod_fid_informe', 'cod_fid_ficha', 'cod_fid_anexo', 'cod_fid_imagen', 
               'cod_fid_informe_en_video', 'cod_fid_informe_multimedia', 'infografia']
new_fid_columns = ['resolucion', 'informe', 'ficha', 'anexo', 'imagen', 'informe_en_video', 'informe_multimedia', 'infografia']

for fid_col, new_fid_col in zip(fid_columns, new_fid_columns):
    df_detalle_Informes[new_fid_col] = df_detalle_Informes[fid_col].apply(lambda x: 'si' if x and x != 'None' else 'no')

# Convertir las columnas a números enteros
for col in ["numero_resolucion", "ano_actuacion", "ventana_de_trabajo"]:
    df_detalle_Informes[col] = df_detalle_Informes[col].fillna(0).astype(int)

# Modificar la columna "auditoria_coordinada"
df_detalle_Informes["auditoria_coordinada"] = df_detalle_Informes["auditoria_coordinada"].apply(lambda x: "si" if x and x != 'None' else "no")

# Modificar las columnas "titulo_difusion" y "archivo_difusion"
for col in ["titulo_difusion", "archivo_difusion"]:
    df_detalle_Informes[col] = df_detalle_Informes[col].apply(lambda x: "si" if x and x != 'None' else "no")

# Seleccionar las columnas requeridas (agregar 'estado')
columnas_seleccionadas = ['codigo_nid', 'titulo','estado', 'gerencia', 'tipo_de_auditoria', 'organismo_auditado', 'actuacion', 
                          'cod_actuacion', 'ano_actuacion', 'ano', 'ventana_de_trabajo', 'periodo_start', 'periodo_end', 
                          'auditoria_coordinada', 'ods', 'sector_publico_nacional', 'objeto_de_auditoria', 'jurisdiccion', 
                          'tags', 'numero_resolucion', 'titulo_difusion', 'archivo_difusion', 'imagen', 'informe_en_video', 
                          'informe_multimedia', 'infografia', 'anexo', 'ficha']

# Crear el DataFrame df_DA_informes
df_DA_informes = df_detalle_Informes[columnas_seleccionadas]

# Llamar a la función para visualizar el DataFrame estilizado
mostrar_dataframe_estilizado(df_DA_informes, num_head=5, num_tail=5)

# Cerrar la conexión a la base de datos
conn.close()

,codigo_nid,titulo,estado,gerencia,tipo_de_auditoria,organismo_auditado,actuacion,cod_actuacion,ano_actuacion,ano,ventana_de_trabajo,periodo_start,periodo_end,auditoria_coordinada,ods,sector_publico_nacional,objeto_de_auditoria,jurisdiccion,tags,numero_resolucion,titulo_difusion,archivo_difusion,imagen,informe_en_video,informe_multimedia,infografia,anexo,ficha
0,12,ENTIDAD BINACIONAL YACYRETÁ SÍNTESIS DE LOS PRINCIPALES MOTIVOS DE LA ABSTENCIÓN DE OPINIÓN DEL INFORME SOBRE LOS ESTADOS FINANCIEROS POR EL EJERCICIO FINALIZADO EL 31/12/2017,Disponible,Gerencia de Control de Gestión del Sector No Financiero,Estados Financieros,ENTIDAD BINACIONAL YACYRETÁ (EBY),400/2018,400,2018,2019,1,2017-01-01,2017-12-31,no,no identificado,no identificado,no identificado,no identificado,"Yacyretá, EBY, Entidad Binacional Yacyretá",8,no,no,no,no,no,no,no,no
1,13,ESTADOS CONTABLES POR EL EJERCICIO FINALIZADO EL 31 DE DICIEMBRE DE 2015 CORRESPONDIENTES A NUCLEOELÉCTRICA ARGENTINA SOCIEDAD ANÓNIMA,Disponible,Gerencia de Control de Gestión del Sector No Financiero,Estados Contables,NUCLEOELÉCTRICA ARGENTINA S.A. (NASA),757/2015,757,2015,2019,4,2015-01-01,2015-12-31,no,no identificado,no identificado,no identificado,no identificado,"Nucleoeléctrica, Argentina , Sociedad , Anónima",10,no,no,no,no,no,no,no,no
2,14,"EVALUAR LA RENDICIÓN DE CUENTAS DEL USO DEL FONDO ANUAL PREVISTO POR EL ARTÍCULO 4°, INCISO D), DE LA LEY N° 19.108, MODIFICADA POR EL ARTÍCULO 73 DE LA LEY N° 26.215, CORRESPONDIENTE AL EJERCICIO 2017",Disponible,Gerencia de Control de Gestión del Sector No Financiero,no identificado,CÁMARA NACIONAL ELECTORAL (CNE),494/2018,494,2018,2019,1,2017-01-01,2017-12-31,no,no identificado,no identificado,no identificado,no identificado,Cámara Nacional Electoral,93,no,no,no,no,no,no,no,no
3,15,"GESTIÓN DEL PROGRAMA 17 “FORMULACIÓN E IMPLEMENTACIÓN DE POLÍTICAS PÚBLICAS DE LA MUJER”, EN LOS ASPECTOS RELACIONADOS A LAS ACCIONES DESARROLLADAS POR EL CONSEJO NACIONAL DE LAS MUJERES (CNM) EN EL MARCO DE LAS PREVISIONES DE LA LEY N° 26.485",Disponible,Gerencia de Control de Gestión del Sector No Financiero,no identificado,"CONSEJO NACIONAL DE LAS MUJERES (CNM), INSTITUTO NACIONAL DE LAS MUJERES (INAM)",670/2016,670,2016,2019,3,2015-01-01,2016-12-31,no,no identificado,no identificado,no identificado,no identificado,"Consejo, Mujeres, CNM, INAM",92,no,no,no,no,no,no,no,si
4,16,MEMORIA INSTITUCIONAL 2018,Disponible,Gerencia de Control de Gestión del Sector No Financiero,no identificado,AUDITORÍA GENERAL DE LA NACIÓN (AGN),630/2018,630,2018,2019,1,2018-01-01,2018-12-31,no,no identificado,no identificado,no identificado,no identificado,"Auditoría General de la Nación, AGN, MEMORIA INSTITUCIONAL",91,no,no,no,no,no,no,no,no
4342,19280,GESTIÓN DE TI SISTEMAS DE INFORMACIÓN - CONTINGENCIA ARSAT,Informe reservado,Gerencia de Control de la Cuenta de Inversión,Informe de Gestión,EMPRESA ARGENTINA DE SOLUCIONES SATELITALES S.A. (ARSAT),487/2021,487,2021,2023,2,2019-09-01,2021-08-31,no,no identificado,no identificado,no identificado,no identificado,"ARSAT, SATÉLITES, SATELITAL",163,no,no,no,no,no,no,no,no
4343,19281,PROGRAMA 16. ASISTENCIA A LA ACTIVIDAD COOPERTATIVA Y MUTUAL,Disponible,Gerencia de Control de Gestión del Sector No Financiero,Informe de Gestión,INSTITUTO NACIONAL DE ASOCIATIVISMO Y ECONOMÍA SOCIAL (INAES),231/2019,231,2019,2023,4,2016-01-01,2019-04-30,no,no identificado,no identificado,no identificado,no identificado,"INAES, COOPERATIVA, MUTUAL, AYUDA",164,no,no,no,no,no,no,no,si
4344,19282,ESTADOS CONTABLES DE CONSTRUCCIÓN DE VIVIENDA PARA LA ARMADA (COVIARA) EJERCICIO 2020.,Disponible,Gerencia de Control de Gestión del Sector No Financiero,Estados Contables,CONSTRUCCIÓN DE VIVIENDA PARA LA ARMADA (COVIARA),92/2022,92,2022,2023,1,2020-01-01,2020-12-31,no,no identificado,no identificado,no identificado,no identificado,no identificado,165,no,no,no,no,no,no,no,no
4345,19283,CERTIFICACIÓN DE LAS TRANSFERENCIAS DE FONDOS DEL ESTADO NACIONAL A AEROLÍNEAS ARGEN

In [4]:
# Guardar el DataFrame en un archivo .csv
df_DA_informes.to_csv('df_DA_informes.csv', index=False)